In [1]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

from SALib.sample import saltelli
from SALib.analyze import sobol


# Label axes, also with units
# Can the indices be negative?


# Aquifer thickness b [m]
b = 10

# Hydraulic Conductivity from Cobuto, 1999 (adapted): 10^-4 - 10^-2 [m/s]
Kmin = 1e-04 * 86400
Kmax = 1e-02 * 86400

# Specific yield Sy from Johnson, 1967: 0.21 - 0.35 [-]
Sy_min = 0.21
Sy_max = 0.35

# Calculating D using eq. (2) from Sawyer et al., 2014
Dmin_temp = (Kmin*b)/Sy_max
Dmax_temp = (Kmax*b)/Sy_min

Dmin = math.log(Dmin_temp, 10) # lower bound for diffusivity
Dmax = math.log(Dmax_temp, 10) # upper bound for diffusivity
phi = 0 # phase

def solution(x, t, A, omega, D, phi):
  """This is the function which gives back the hydraulic head
  omega: frequency
  A: amplitude
  x: distance
  t: time"""
  sinus = math.sin(-x*math.sqrt(omega/(2*D))+ omega*t + phi)
  ret = A*math.exp(-x*math.sqrt(omega/(2*D)))*sinus
  return ret 




In [42]:
class Result:
   def __repr__(self):
    return 'D={}, A={}, w={}, x={}, t={}, hxt={}'.format(
      self.D,
      self.A,
      self.w,
      self.x,
      self.t,
      self.hxt,        
    )

def calculate_hxt_for_one_variable(D_input, A_input, w_input, x_input, t_input):
  fun1 = A_input * math.exp(-x_input * math.sqrt(w_input / (2*D_input)))
  fun2 = math.sin(-x_input * math.sqrt((w_input / (2*D_input)) + ((w_input * t_input) )))
  return fun1 * fun2

In [48]:
# Define a general function to generate n different realizations of the hydraulic head.
def calculate_hxt(x_vector, t_vector, A_vector, w_vector, D_vector):
	results = []

	for simulation_index in range(0, TOTAL_NUMBER_OF_SIMULATIONS):
		temp_result = Result()

		for x_index in range(0, len(x_vector)):
			for t_index in range(0, len(t_vector)):
				temp_result = Result()				
				temp_result.D = D_vector[simulation_index]
				temp_result.A = A_vector[simulation_index]
				temp_result.w = w_vector[simulation_index]
				temp_result.x = x_vector[x_index]
				temp_result.t = t_vector[t_index]
				temp_result.hxt = solution(temp_result.x, temp_result.t,temp_result.A, temp_result.w,  temp_result.D, phi)
				#temp_result.hxt = calculate_hxt_for_one_variable(temp_result.D, temp_result.A, temp_result.w, temp_result.x, temp_result.t)
				results.append(temp_result.hxt)

	return results

In [49]:
num_of_simulations = 10000

# vectors with 10000 samples of the parameters in matrix A & B
D_vector_A = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_A = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_A = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]

print(statistics.mean(D_vector_A))
print(statistics.mean(A_vector_A))
print(statistics.mean(omega_vector_A))



D_vector_B = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_B = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_B = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]

7945.180877311971
0.3855004979778874
2.022225396950915


In [50]:
def calculate_solution(x, t, A_vector, omega_vector, D_vector):
    # return 10000 simulations of the hydraulic head
    y = []

    for idx in range(num_of_simulations):
        hydraulic_head = solution(x, t, A_vector[idx], omega_vector[idx], D_vector[idx], phi)
        y.append(hydraulic_head)
  
    return y

In [55]:
distances = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days

import statistics

TOTAL_NUMBER_OF_SIMULATIONS = 10000

for x in distances:
    for t in timesteps:

        y_A = calculate_solution(x, t, A_vector_A, omega_vector_A, D_vector_A)
        y_B = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_B)
        y_C_1 = calculate_solution(x, t, A_vector_A, omega_vector_B, D_vector_B)
        y_C_2 = calculate_solution(x, t, A_vector_B, omega_vector_A, D_vector_B)
        y_C_3 = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_A)


        avg = (sum(y_A)/num_of_simulations) * (sum(y_B)/num_of_simulations)
        print(avg)
        
        # First order sensitivity indices
        S_1 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_2 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_3 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)

        print('S_A='+str(S_1))
        print('S_omega='+str(S_2))
        print('S_D='+str(S_3))
        print(" ")

0.0021193041927398838
S_A=0.027941285625285653
S_omega=0.6986768613897302
S_D=0.01832442640315155
 
1.867158725790679e-05
S_A=0.0033640467097938724
S_omega=0.7161772328923017
S_D=0.0077254027562441725
 
3.934947209639036e-06
S_A=-0.02739965193147865
S_omega=0.7200161422795919
S_D=-0.009765864708170402
 
0.001345784514752668
S_A=0.032447673770399976
S_omega=0.6681737338700149
S_D=0.020355956324925397
 
6.858668186703359e-06
S_A=0.004944371473340439
S_omega=0.6856981603032151
S_D=0.006514668676603486
 
-3.567672674589156e-07
S_A=-0.023182738836143833
S_omega=0.6856531376653113
S_D=-0.006790679938136604
 
3.278746142604365e-05
S_A=-0.003131656597817001
S_omega=0.2958513007790431
S_D=0.02257848810023752
 
-1.3814502292553339e-06
S_A=0.009881932790706666
S_omega=0.2983528022567091
S_D=0.014283572656811551
 
-1.2984987073556988e-06
S_A=-0.015580126280666465
S_omega=0.3006458390308325
S_D=-0.005894147495209085
 


In [56]:
inpt = {
    'num_vars': 3,
    'names': ['A', 'w', 'D'],
    'bounds': [[10**(-1) , 1],
               [(2*math.pi)/7, (2*math.pi)/2],
               [10**Dmin, 10**Dmax]],
    'dists': ['unif', 'unif', 'unif'] 
}

param_values = saltelli.sample(inpt, N = 2**13, calc_second_order=True)

C:\Users\z004n49n\AppData\Local\Temp\ipykernel_15968\4211220531.py:10: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(inpt, N = 2**13, calc_second_order=True)


In [57]:
def return_hydraulic_head(values, x_input, t_input):
    Y = np.zeros([values.shape[0]])
    for i,X in enumerate(values):
          Y[i] = (X[0] * math.exp(-x_input * math.sqrt(X[1] / (2*X[2])))) * (math.sin(-x_input * math.sqrt(X[1] / (2*X[2])) + ((X[1] * t_input))))  
    return Y

In [58]:
for x in distances:
    for t in timesteps:
        
        Y = return_hydraulic_head(param_values, x_input = np.array([x]), t_input = np.array([t]))
        sobol.analyze(inpt, Y, print_to_console=True)

         ST   ST_conf
A  0.187392  0.005687
w  0.994485  0.032765
D  0.000079  0.000008
         S1   S1_conf
A  0.006209  0.014352
w  0.813115  0.025905
D -0.000004  0.000288
              S2   S2_conf
(A, w)  0.181223  0.022996
(A, D) -0.000002  0.024733
(w, D)  0.000039  0.026299
         ST   ST_conf
A  0.182562  0.006614
w  1.003610  0.027098
D  0.000084  0.000010
         S1   S1_conf
A -0.005315  0.013417
w  0.826339  0.023261
D -0.000173  0.000235
              S2   S2_conf
(A, w)  0.187888  0.021472
(A, D)  0.010564  0.024681
(w, D)  0.000102  0.021771
         ST   ST_conf
A  0.182512  0.006576
w  0.948832  0.029939
D  0.000077  0.000008
         S1   S1_conf
A  0.009402  0.012546
w  0.774192  0.026218
D -0.000072  0.000261
              S2   S2_conf
(A, w)  0.173808  0.021875
(A, D) -0.000055  0.022275
(w, D)  0.002118  0.021967
         ST   ST_conf
A  0.187426  0.006916
w  0.994044  0.033581
D  0.006445  0.000625
         S1   S1_conf
A  0.006210  0.013332
w  0.807621  0.0